In [52]:
#==================================================
#                Load overlay
#=================================================
from pynq import Overlay
overlay = Overlay("design.bit")
overlay?

In [53]:
#==================================================
#                Load sha256 IP
#=================================================
sha256_ip = overlay.sha2560
sha256_ip?
sha256_ip.mmio
overlay.ip_dict

{'sha2560': {'addr_range': 65536,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'sha2560',
  'gpio': {},
  'interrupts': {},
  'mem_id': 's_axi_ctrl_bus',
  'parameters': {'C_M_AXI_INTPUT_ADDR_WIDTH': '32',
   'C_M_AXI_INTPUT_ARUSER_WIDTH': '1',
   'C_M_AXI_INTPUT_AWUSER_WIDTH': '1',
   'C_M_AXI_INTPUT_BUSER_WIDTH': '1',
   'C_M_AXI_INTPUT_CACHE_VALUE': '"0011"',
   'C_M_AXI_INTPUT_DATA_WIDTH': '32',
   'C_M_AXI_INTPUT_ENABLE_ID_PORTS': 'false',
   'C_M_AXI_INTPUT_ENABLE_USER_PORTS': 'false',
   'C_M_AXI_INTPUT_ID_WIDTH': '1',
   'C_M_AXI_INTPUT_PROT_VALUE': '"000"',
   'C_M_AXI_INTPUT_RUSER_WIDTH': '1',
   'C_M_AXI_INTPUT_USER_VALUE': '0x00000000',
   'C_M_AXI_INTPUT_WUSER_WIDTH': '1',
   'C_M_AXI_OUTPUT_R_ADDR_WIDTH': '32',
   'C_M_AXI_OUTPUT_R_ARUSER_WIDTH': '1',
   'C_M_AXI_OUTPUT_R_AWUSER_WIDTH': '1',
   'C_M_AXI_OUTPUT_R_BUSER_WIDTH': '1',
   'C_M_AXI_OUTPUT_R_CACHE_VALUE': '"0011"',
   'C_M_AXI_OUTPUT_R_DATA_WIDTH': '32',
   'C_M_AXI_OUTPUT_R_ENABLE_ID_PORTS': 'false',
   'C

In [54]:
#===============================================
#                  load the array
#==============================================
from pynq import Xlnk
import numpy
xlnk = Xlnk()

string = "hashcat"
input_data = bytearray(string,'utf-16')
output_size = 32
input_buffer = xlnk.cma_array(shape=(len(input_data),), dtype=numpy.uint32)
in_buffer_address=input_buffer.physical_address
output_buffer = xlnk.cma_array(shape=(output_size,), dtype=numpy.uint32)
out_buffer_address=output_buffer.physical_address
print(type(input_data))

<class 'bytearray'>


In [55]:
#===============================================
#           write image to memroy
#==============================================

numpy.copyto(input_buffer,input_data)
print(input_buffer)
print(type(input_buffer))


from pynq import MMIO
mmio = MMIO(in_buffer_address,len(input_data))


[255 254 104   0  97   0 115   0 104   0  99   0  97   0 116   0]
<class 'pynq.xlnk.ContiguousArray'>


In [56]:
#===============================================
#                  sha256 hw Ex
#==============================================
from pynq import MMIO
       

#write data
byte_framein_offset = 1920
nrows               = 1080
ncol                = 1920


#address
IP_BASE_ADDRESS    =  0x43C00000
ADDRESS_RANGE      =  64000
# #address offset
# ADDR_AP_CRTL       =  0x00
# ADDR_GIE           =  0x04
# ADDR_IER           =  0x08
# ADDR_ISR           =  0x0c

# ADDR_ROWS          =  0x24       
# ADDR_COLS          =  0x2c
# ADDR_RDOFFSET       = 0x14
# ADDR_WROFFSET       = 0x1c
# ADDR_FRAMEIN_OFFSET = 0x34


XSHA256_ENCRYPT_CTRL_BUS_ADDR_AP_CTRL  = 0x00
XSHA256_ENCRYPT_CTRL_BUS_ADDR_GIE = 0x04
XSHA256_ENCRYPT_CTRL_BUS_ADDR_IER = 0x08
XSHA256_ENCRYPT_CTRL_BUS_ADDR_ISR = 0x0c
XSHA256_ENCRYPT_CTRL_BUS_ADDR_SUPWD_DATA = 0x10
XSHA256_ENCRYPT_CTRL_BUS_BITS_SUPWD_DATA = 32
XSHA256_ENCRYPT_CTRL_BUS_ADDR_DIGEST_DATA = 0x18
XSHA256_ENCRYPT_CTRL_BUS_BITS_DIGEST_DATA = 32

# sobel mapping memory
def SHA_Init_EX():
    # mapping memory

    mmio = MMIO(IP_BASE_ADDRESS,ADDRESS_RANGE)
    mmio.write(XSHA256_ENCRYPT_CTRL_BUS_ADDR_SUPWD_DATA,in_buffer_address)
    mmio.write(XSHA256_ENCRYPT_CTRL_BUS_ADDR_DIGEST_DATA,out_buffer_address)
    
#     print("inbuff_address:",mmio.read(ADDR_RDOFFSET))
#     print("out_buffer_address:",mmio.read(ADDR_WROFFSET))

    
#Start Or Stop the FPGA IP Core   start = 1; stop = 0;       
def SHA_StartStop_Ex(state):
    # mapping memory
    mmio = MMIO(IP_BASE_ADDRESS,ADDRESS_RANGE )
    if(state):
        #enable the interrupt
        #enable the interrupt global
        mmio.write(XSHA256_ENCRYPT_CTRL_BUS_ADDR_GIE,1)
        mmio.write(XSHA256_ENCRYPT_CTRL_BUS_ADDR_IER,1)
        
        #Start the filter
        mmio.write(XSHA256_ENCRYPT_CTRL_BUS_ADDR_AP_CTRL,0x1)
    else:
        while True:
            ap_done =  (mmio.read(XSHA256_ENCRYPT_CTRL_BUS_ADDR_AP_CTRL)>>1)&0x01
            if(ap_done):
                break
                
        mmio.write(XSHA256_ENCRYPT_CTRL_BUS_ADDR_GIE,0x0)
        mmio.write(XSHA256_ENCRYPT_CTRL_BUS_ADDR_IER,0x0)

In [68]:
#                  sha256 hw Ex  
#==============================================
import numpy
import time
#Set Control Registers State Or Value
SHA_Init_EX() 
time.sleep(3)

#Start The FPGA IP Core
start = time.time()
SHA_StartStop_Ex(1)
#time.sleep(4)
#Stop The FPGA IP Core check Whether The IP Core is End
SHA_StartStop_Ex(0)
stop = time.time()
time_sha256_fpga = stop - start
print("Sobel FPGA time:",time_sha256_fpga)
temp = output_buffer.copy()

for i in range(0,8):
    print(hex(temp[i]), end = '')

Sobel FPGA time: 1.3818387985229492
0xb12605320xa20d7eb70xf91026230x7c7cdf6a0xf357810d0x866484260xa46549040x53a8a149